In [1]:
#Data Processing for the Primary Questions

In [2]:
# Necessary libraries were called
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from rapidfuzz import process

import csv
import string
from sqlalchemy import create_engine
import sqlite3

In [3]:
#Here we have call the csv files and reseted the index
res_2014 = pd.read_csv('constituency_wise_results_2014.csv')
res_2014.reset_index(drop = True, inplace = True)
res_2019 = pd.read_csv('constituency_wise_results_2019.csv')
res_2019.reset_index(drop = True, inplace = True)

di_states = pd.read_csv('dim_states_codes.csv')

In [4]:
#Here we check the state Telangana and channge specific constitencies in the year 2014
res_2014[res_2014['state'] == 'Telangana']

,state,pc_name,candidate,sex,age,category,party,party_symbol,general_votes,postal_votes,total_votes,total_electors


In [5]:
#
res_2019['party_symbol'].fillna('NOTA', inplace = True)
res_2019

,state,pc_name,candidate,sex,age,category,party,party_symbol,general_votes,postal_votes,total_votes,total_electors
0,Andhra Pradesh,Aruku,KISHORE CHANDRA DEO,MALE,72.0,ST,TDP,Bicycle,336163,1938,338101,1451418
1,Andhra Pradesh,Aruku,Dr. KOSURI KASI VISWANADHA VEERA VENKATA SATYA...,MALE,54.0,ST,BJP,Lotus,17578,289,17867,1451418
2,Andhra Pradesh,Aruku,GODDETI. MADHAVI,FEMALE,26.0,ST,YSRCP,Ceiling Fan,557561,4629,562190,1451418
3,Andhra Pradesh,Aruku,SHRUTI DEVI VYRICHERLA,FEMALE,46.0,ST,INC,Hand,17656,74,17730,1451418
4,Andhra Pradesh,Aruku,GANGULAIAH VAMPURU.,MALE,49.0,ST,JnP,Glass Tumbler,42245,549,42794,1451418
...,...,...,...,...,...,...,...,...,...,...,...,...
8592,Puducherry,Puducherry,TIRAVITAMANKAI@LOURDHUMARY,FEMALE,57.0,GENERAL,IND,Slate,870,0,870,973410
8593,Puducherry,Puducherry,IMS. BALAJI,MALE,32.0,GENERAL,IND,Bat,2546,0,2546,973410
8594,Puducherry,Puducherry,RAMESH.D,MALE,49.0,SC,IND,Mike,1653,0,1653,973410
8595,Puducherry,Puducherry,V. RAMAMURTHI,MALE,74.0,GENERAL,IND,Whistle,641,1,642,973410


In [6]:
Const_1 = res_2014['candidate'].dropna().unique()
Const_2 = res_2019['candidate'].dropna().unique()

identical_Const = pd.Series(list(set(Const_1).intersection(set(Const_2))))

identical_Const

0      AVTAR SINGH BHADANA
1           YOGENDRA YADAV
2           RAJ KR. PASWAN
3           HARPREET SINGH
4      MANGAL KUMAR SARDAR
              ...         
519     SUSHIL KUMAR SINGH
520          KANTE SAYANNA
521         SHANKAR JADHAV
522       P.V.MIDHUN REDDY
523             ARUN KUMAR
Length: 524, dtype: object

In [7]:
def find_similar_matches(Const_1, Const_2, threshold=80):
    matches = []
    for constituency in Const_1:
        match_tuple = process.extractOne(constituency, Const_2, score_cutoff=threshold)
        if match_tuple:  # Ensure match_tuple is not None
            match, score = match_tuple[:2]  # Extract match and score
            if score >= threshold:
                matches.append((constituency, match, score))
    return matches

In [8]:
similar_matches = find_similar_matches(Const_1, Const_2)

for constituency1, match, score in similar_matches:
    if constituency1 not in identical_Const.values:
        # Find indices in df1 where constituency matches
        indices_to_update = res_2014[res_2014['candidate'] == constituency1].index
        # Update each row in df1 with the correct constituency name
        for idx in indices_to_update:
            res_2014.at[idx, 'candidate'] = match

In [9]:
Const_3 = res_2014['candidate'].dropna().unique()
Const_4 = res_2019['candidate'].dropna().unique()

identical_Const = pd.Series(list(set(Const_3).intersection(set(Const_4))))

In [10]:
def find_similar_matches(Const_3, Const_4, threshold=80):
    matches = []
    for constituency in Const_1:
        match_tuple = process.extractOne(constituency, Const_4, score_cutoff=threshold)
        if match_tuple:  # Ensure match_tuple is not None
            match, score = match_tuple[:2]  # Extract match and score
            if score >= threshold:
                matches.append((constituency, match, score))
    return matches

similar_matches = find_similar_matches(Const_3, Const_4)

In [11]:
Const_5 = res_2014['candidate'].dropna().unique()
Const_6 = res_2019['candidate'].dropna().unique()

identical_Const = pd.Series(list(set(Const_5).intersection(set(Const_6))))

In [12]:
def find_similar_matches(Const_5, Const_6, threshold=80):
    matches = []
    for constituency in Const_1:
        match_tuple = process.extractOne(constituency, Const_2, score_cutoff=threshold)
        if match_tuple:  # Ensure match_tuple is not None
            match, score = match_tuple[:2]  # Extract match and score
            if score >= threshold:
                matches.append((constituency, match, score))
    return matches

similar_matches = find_similar_matches(Const_5, Const_6)

In [13]:
st = res_2014[res_2014['state'] == 'Andhra Pradesh']
constituencies_to_transfer = st['candidate']

constituencies_to_transfer

res_2014.loc[res_2014['state'].isin(constituencies_to_transfer), 'state'] = 'Andhra Pradesh'
res_2014['state'] = res_2014['state'].replace({'Andhra Pradesh': 'Telangana'})

In [14]:
states_to_drop_2019 = res_2019['state'].value_counts()
states_to_drop_2019 = states_to_drop_2019[states_to_drop_2019 == 0].index.tolist()

states_to_2014_drop = res_2014['state'].value_counts()
states_to_2014_drop = states_to_2014_drop[states_to_2014_drop == 0].index.tolist()

In [15]:
res_2014 = res_2014[~res_2014['state'].isin(states_to_2014_drop)]
res_2019 = res_2019[~res_2019['state'].isin(states_to_drop_2019)]

In [16]:
Party_Symbol_2014_ID = pd.DataFrame({'party_symbol' : res_2014['party_symbol'].unique()})
Party_Symbol_2014_ID['PS_ID'] = Party_Symbol_2014_ID['party_symbol'].str[:1].str.upper() + Party_Symbol_2014_ID['party_symbol'].str[-1].str.upper() + pd.RangeIndex(start = 1, stop = (len(Party_Symbol_2014_ID['party_symbol']) + 1), step = 1).astype(str)
Party_Symbol_2014_ID.set_index('party_symbol', inplace = True)

Party_Symbol_2019_ID = pd.DataFrame({'party_symbol' : res_2019['party_symbol'].unique()})
Party_Symbol_2019_ID['PS_ID'] = Party_Symbol_2019_ID['party_symbol'].str[:1].str.upper() + Party_Symbol_2019_ID['party_symbol'].str[-1].str.upper() + pd.RangeIndex(start = 1, stop = (len(Party_Symbol_2019_ID['party_symbol']) + 1), step = 1).astype(str)
Party_Symbol_2019_ID.set_index('party_symbol', inplace = True)

In [17]:
def is_special_character(char):
    
    return char in string.punctuation

def extract_first_characters(name):
    
    words = name.split()
    first_characters = []
    for word in words:
        if len(word) > 1 and is_special_character(word[0]):
            first_characters.append(word[1])
        else:
            first_characters.append(word[0])
    
    return ''.join(first_characters)

In [18]:
Candidate_2014_ID = pd.DataFrame({'candidate' : res_2014['candidate'].unique()})
Candidate_2014_ID['Can_ID'] = Candidate_2014_ID['candidate'].apply(lambda name: extract_first_characters(name) if pd.notnull(name) else '') 
Candidate_2014_ID.set_index('candidate', inplace = True)

Candidate_2019_ID = pd.DataFrame({'candidate' : res_2019['candidate'].unique()})
Candidate_2019_ID['Can_ID'] = Candidate_2019_ID['candidate'].apply(lambda name: extract_first_characters(name) if pd.notnull(name) else '') 
Candidate_2019_ID.set_index('candidate', inplace = True)

In [19]:
PC_name_2014_ID = pd.DataFrame({'pc_name' : res_2014['pc_name'].unique()})
PC_name_2014_ID['PCN_ID'] = PC_name_2014_ID['pc_name'].apply(lambda name: extract_first_characters(name) if pd.notnull(name) else '') + pd.RangeIndex(start = 1, stop = (len(PC_name_2014_ID['pc_name']) + 1), step = 1).astype(str)
PC_name_2014_ID.set_index('pc_name', inplace = True)

PC_name_2019_ID = pd.DataFrame({'pc_name' : res_2019['pc_name'].unique()})
PC_name_2019_ID['PCN_ID'] = PC_name_2019_ID['pc_name'].apply(lambda name: extract_first_characters(name) if pd.notnull(name) else '') + pd.RangeIndex(start = 1, stop = (len(PC_name_2019_ID['pc_name']) + 1), step = 1).astype(str)
PC_name_2019_ID.set_index('pc_name', inplace = True)

In [20]:
res_2014_DB = pd.merge(res_2014, Party_Symbol_2014_ID, how = 'left', on = 'party_symbol')
res_2019_DB = pd.merge(res_2019, Party_Symbol_2019_ID, how = 'left', on = 'party_symbol')

res_2014_DB = pd.merge(res_2014_DB, Candidate_2014_ID, how = 'left', on = 'candidate')
res_2019_DB = pd.merge(res_2019_DB, Candidate_2019_ID, how = 'left', on = 'candidate')

In [21]:
di_states.rename(columns =  {'state_name' : 'state'}, inplace = True)

In [22]:
res_2014_DB = pd.merge(res_2014_DB, di_states, how = 'left', on = 'state')
res_2019_DB = pd.merge(res_2019_DB, di_states, how = 'left', on = 'state')

res_2014_DB = pd.merge(res_2014_DB, PC_name_2014_ID, how = 'left', on = 'pc_name')
res_2019_DB = pd.merge(res_2019_DB, PC_name_2019_ID, how = 'left', on = 'pc_name')

In [23]:
res_2014_DB['ID'] = "14" + res_2014_DB['Can_ID'] + "_" + res_2014_DB['abbreviation'] + res_2014_DB['PCN_ID'] + "_" + res_2014_DB['PS_ID'] + res_2014_DB['party']
res_2014_DB.set_index('ID', inplace = True)

res_2019_DB['ID'] = "19" + res_2019_DB['Can_ID'] + "_" + res_2019_DB['abbreviation'] + res_2019_DB['PCN_ID'] + "_" + res_2019_DB['PS_ID'] + res_2019_DB['party']
res_2019_DB.set_index('ID', inplace = True)

In [24]:
res_2014_DB

,state,pc_name,candidate,sex,age,category,party,party_symbol,general_votes,postal_votes,total_votes,total_electors,PS_ID,Can_ID,abbreviation,PCN_ID
ID,,,,,,,,,,,,,,,,
14NSB_TGA1_CR1TRS,Telangana,Adilabad,NAGESH SUBHASH BOBATE,M,49.0,ST,TRS,Car,425762,5085,430847,1386282,CR1,NSB,TG,A1
14NKP_TGA1_HD2INC,Telangana,Adilabad,NARESH KUMAR POOJALA,M,37.0,ST,INC,Hand,257994,1563,259557,1386282,HD2,NKP,TG,A1
14RR_TGA1_BE3TDP,Telangana,Adilabad,RATHOD RAMESH,M,48.0,ST,TDP,Bicycle,182879,1319,184198,1386282,BE3,RR,TG,A1
14RGB_TGA1_ET4BSP,Telangana,Adilabad,RATHOD GOVINDBHAI BHIKHABHAI,M,55.0,ST,BSP,Elephant,94363,57,94420,1386282,ET4,RGB,TG,A1
14NR_TGA1_AW5IND,Telangana,Adilabad,NETHAWATH RAMDAS,M,44.0,ST,IND,Auto- Rickshaw,41028,4,41032,1386282,AW5,NR,TG,A1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14E_PYP508_SE11IND,Puducherry,Puducherry,ESWARA,M,60.0,GEN,IND,Sewing Machine,465,0,465,901357,SE11,E,PY,P508
14KPR_PYP508_CT23IND,Puducherry,Puducherry,K. PRABHAKARA REDDY,M,41.0,GEN,IND,Coconut,767,0,767,901357,CT23,KPR,PY,P508
14JRP_PYP508_CD25IND,Puducherry,Puducherry,Jerambhai R. Parmar,M,40.0,GEN,IND,Carrom Board,497,0,497,901357,CD25,JRP,PY,P508


In [25]:
res_2019_DB

,state,pc_name,candidate,sex,age,category,party,party_symbol,general_votes,postal_votes,total_votes,total_electors,PS_ID,Can_ID,abbreviation,PCN_ID
ID,,,,,,,,,,,,,,,,
19KCD_APA1_BE1TDP,Andhra Pradesh,Aruku,KISHORE CHANDRA DEO,MALE,72.0,ST,TDP,Bicycle,336163,1938,338101,1451418,BE1,KCD,AP,A1
19DKKVVVSR_APA1_LS2BJP,Andhra Pradesh,Aruku,Dr. KOSURI KASI VISWANADHA VEERA VENKATA SATYA...,MALE,54.0,ST,BJP,Lotus,17578,289,17867,1451418,LS2,DKKVVVSR,AP,A1
19GM_APA1_CN3YSRCP,Andhra Pradesh,Aruku,GODDETI. MADHAVI,FEMALE,26.0,ST,YSRCP,Ceiling Fan,557561,4629,562190,1451418,CN3,GM,AP,A1
19SDV_APA1_HD4INC,Andhra Pradesh,Aruku,SHRUTI DEVI VYRICHERLA,FEMALE,46.0,ST,INC,Hand,17656,74,17730,1451418,HD4,SDV,AP,A1
19GV_APA1_GR5JnP,Andhra Pradesh,Aruku,GANGULAIAH VAMPURU.,MALE,49.0,ST,JnP,Glass Tumbler,42245,549,42794,1451418,GR5,GV,AP,A1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19T_PYP540_SE114IND,Puducherry,Puducherry,TIRAVITAMANKAI@LOURDHUMARY,FEMALE,57.0,GENERAL,IND,Slate,870,0,870,973410,SE114,T,PY,P540
19IB_PYP540_BT21IND,Puducherry,Puducherry,IMS. BALAJI,MALE,32.0,GENERAL,IND,Bat,2546,0,2546,973410,BT21,IB,PY,P540
19R_PYP540_ME6IND,Puducherry,Puducherry,RAMESH.D,MALE,49.0,SC,IND,Mike,1653,0,1653,973410,ME6,R,PY,P540


In [26]:
resultant = pd.concat([res_2014_DB, res_2019_DB], ignore_index = False)

In [27]:
resultant

,state,pc_name,candidate,sex,age,category,party,party_symbol,general_votes,postal_votes,total_votes,total_electors,PS_ID,Can_ID,abbreviation,PCN_ID
ID,,,,,,,,,,,,,,,,
14NSB_TGA1_CR1TRS,Telangana,Adilabad,NAGESH SUBHASH BOBATE,M,49.0,ST,TRS,Car,425762,5085,430847,1386282,CR1,NSB,TG,A1
14NKP_TGA1_HD2INC,Telangana,Adilabad,NARESH KUMAR POOJALA,M,37.0,ST,INC,Hand,257994,1563,259557,1386282,HD2,NKP,TG,A1
14RR_TGA1_BE3TDP,Telangana,Adilabad,RATHOD RAMESH,M,48.0,ST,TDP,Bicycle,182879,1319,184198,1386282,BE3,RR,TG,A1
14RGB_TGA1_ET4BSP,Telangana,Adilabad,RATHOD GOVINDBHAI BHIKHABHAI,M,55.0,ST,BSP,Elephant,94363,57,94420,1386282,ET4,RGB,TG,A1
14NR_TGA1_AW5IND,Telangana,Adilabad,NETHAWATH RAMDAS,M,44.0,ST,IND,Auto- Rickshaw,41028,4,41032,1386282,AW5,NR,TG,A1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19T_PYP540_SE114IND,Puducherry,Puducherry,TIRAVITAMANKAI@LOURDHUMARY,FEMALE,57.0,GENERAL,IND,Slate,870,0,870,973410,SE114,T,PY,P540
19IB_PYP540_BT21IND,Puducherry,Puducherry,IMS. BALAJI,MALE,32.0,GENERAL,IND,Bat,2546,0,2546,973410,BT21,IB,PY,P540
19R_PYP540_ME6IND,Puducherry,Puducherry,RAMESH.D,MALE,49.0,SC,IND,Mike,1653,0,1653,973410,ME6,R,PY,P540


In [28]:
total_votes_by_candidate = resultant.groupby('candidate')['total_votes'].sum().reset_index()
total_votes_by_pc_name = resultant.groupby(['pc_name', 'candidate'])['total_votes'].sum().reset_index()
votes_by_candidate_and_state = resultant.groupby(['state', 'candidate'])['total_votes'].sum().reset_index()
votes_by_candidate_and_party= resultant.groupby(['party', 'candidate'])['total_votes'].sum().reset_index()
votes_by_candidate_and_party_symbol = resultant.groupby(['party_symbol', 'candidate'])['total_votes'].sum().reset_index()

In [29]:
print("Votes for Candidate: \n", total_votes_by_candidate)
print("Votes for Constituncies: \n",total_votes_by_pc_name)
print("Votes for state: \n",votes_by_candidate_and_state)
print("Votes for party: \n", votes_by_candidate_and_party)
print("Votes for party symbol: \n", votes_by_candidate_and_party_symbol)

Votes for Candidate: 
                        candidate  total_votes
0        "YOGESH NANAJI SAMRITE"        46345
1          (CA) SANJAY KUNDALIYA         1098
2           (KUNWAR) ROBIN TYAGI         2056
3               108 NILESH DAMOR         8496
4              A M Izharul Haque         1842
...                          ...          ...
9209                Zaffar Obaid        17583
9210               Zakir Hussain         4315
9211   Zambad Subhash Manakchand        91789
9212  Zitruji Chandruji Borutkar         1318
9213        Znyosho Vijayprakash          298

[9214 rows x 2 columns]
Votes for Constituncies: 
               pc_name                                candidate  total_votes
0      ARUNACHAL EAST                              Bandey Mili        15958
1      ARUNACHAL EAST                            C. C. Singpho        11493
2      ARUNACHAL EAST                        Lowangcha Wanglat        83935
3      ARUNACHAL EAST                             Mongol Yomso       

In [30]:
#Secondary Questions

In [31]:
res_2014_DB.to_csv('resultant_2014.csv', index = True)
res_2019_DB.to_csv('resultant_2019.csv', index = True)
resultant.to_csv('resultant.csv', index = True)

In [32]:
engine = create_engine('sqlite://', echo = False)

In [33]:
res_2014.to_sql('res_2014', con = engine, index = False, if_exists = 'replace')

8355

In [34]:
query_1 = """
SELECT 
    candidate, 
    (SUM(total_votes) * 100.000 / SUM(total_electors)) AS Voters_turnout_per,
    (SUM(postal_votes) * 100.000 / SUM(total_electors)) AS Postal_turnout_per
FROM res_2014
GROUP BY candidate
ORDER BY Voters_turnout_per DESC
"""

In [35]:
result = pd.read_sql(query_1, engine)

In [36]:
cor = result['Postal_turnout_per'].corr(result['Voters_turnout_per'])

In [37]:
if 'Postal_turnout_per' in result.columns and 'Voters_turnout_per' in result.columns :
    print(f'Correlation between Postal votes percentage and Voter Turnout percentage: {cor}')
else :
    print('Columns for calculating correlation nit found in the result set.')

Correlation between Postal votes percentage and Voter Turnout percentage: 0.46132759368256315


In [38]:
GDP = pd.read_excel('TABLE25_17-23.xlsx')
GDP_TB27 = pd.read_excel('TABLE27_17-23.xlsx')

In [39]:
GDP.drop('Unnamed: 0', axis = 1, inplace = True)
GDP.rename(columns = {'Unnamed: 1' : 'State', 'Unnamed: 2' : '2017-18', 'Unnamed: 3' : '2018-19', 'Unnamed: 4' : '2019-20', 'Unnamed: 5' : '2020-21',
                    'Unnamed: 6' : '2021-22', 'Unnamed: 7' : '2022-23'}, inplace = True)
GDP.drop([0, 1, 2, 3, 39, 40], axis = 0, inplace = True)
GDP.drop([4], axis= 0, inplace = True)
GDP.fillna('0', inplace = True)
GDP.replace('-', 0, inplace = True)

In [40]:
GDP

,State,2017-18,2018-19,2019-20,2020-21,2021-22,2022-23
5,Andaman & Nicobar Islands,178709,204254,219653,205368,229080,0
6,Andhra Pradesh,138299,154031,160341,163746,192587,219518
7,Arunachal Pradesh,138836,155103,182171,181684,205645,0
8,Assam,75151,81034,90123,90482,102965,118504
9,Bihar,36850,40715,44175,42083,47498,54111
10,Chandigarh,280512,307812,330703,290671,333932,0
11,Chhattisgarh,88793,102024,106603,104788,120704,133898
12,Delhi,318323,338730,355798,331112,389529,444768
13,Goa,411740,423716,435949,431351,472070,0
14,Gujarat,176961,197457,212428,207324,241930,0


In [41]:
GDP_TB27.drop('Unnamed: 0', axis = 1, inplace = True)
GDP_TB27.rename(columns = {'TABLE 27: GROSS STATE DOMESTIC PRODUCT' : 'State', 'Unnamed: 2' : '2017-18', 'Unnamed: 3' : '2018-19', 'Unnamed: 4' : '2019-20', 'Unnamed: 5' : '2020-21',
                    'Unnamed: 6' : '2021-22', 'Unnamed: 7' : '2022-23'}, inplace = True)
GDP_TB27.drop([0, 1, 2, 3, 38, 39], axis = 0, inplace = True)
GDP_TB27.drop([4], axis= 0, inplace = True)
GDP_TB27.fillna('0', inplace = True)
GDP_TB27.replace('-', 0, inplace = True)

In [42]:
GDP_TB27

,State,2017-18,2018-19,2019-20,2020-21,2021-22,2022-23
5,Andhra Pradesh,78613542,87372111,92583912,95678770,113383650,131772815
6,Arunachal Pradesh,2247478,2533487,3002365,3054781,3512354,0
7,Assam,28316489,30933632,34685068,35360542,41261187,49316660
8,Bihar,46874631,52797582,58185548,56726250,65030243,75139559
9,Chandigarh,3634435,4013887,4342143,3944204,4563526,0
10,Chhattisgarh,28273744,32710666,34464813,34775246,40641582,45760826
11,Delhi,67790004,73838943,79291127,76343541,90464204,104375934
12,Goa,6935205,7185334,7503209,7570540,8260370,0
13,Gujarat,132909477,149215571,161714320,161610636,193706646,0
14,Haryana,63883208,69893976,73219451,74185007,87066453,99415408


In [43]:
res_2019.to_sql('res_2019', con = engine, index = False, if_exists = 'replace')

8597

In [44]:
query_2 = """
SELECT DISTINCT
    state, 
    (SUM(total_votes) * 100.000 / SUM(total_electors)) AS Voters_turnout_per
FROM res_2019
GROUP BY state
ORDER BY Voters_turnout_per DESC
"""

In [45]:
result_1 = pd.read_sql(query_2, engine)
GDP_cor = result_1['Voters_turnout_per'].corr(GDP_TB27['2018-19'])

In [46]:
if '2018-19' in GDP_TB27.columns and 'Voters_turnout_per' in result_1.columns :
    print(f'Correlation between Votes on GDP percentage and Voter Turnout percentage: {GDP_cor}')
else :
    print('Columns for calculating correlation nit found in the result set.')

Correlation between Votes on GDP percentage and Voter Turnout percentage: -0.1548302235178507


In [47]:
Literacy_11 = pd.read_excel('TABLE01_1991-2011.xlsx')
Literacy_23 = pd.read_excel('TABLE00_2011-23.xlsx')

In [48]:
Literacy_11['Avg_Urban_2011'] =  (Literacy_11['Male_Urban_2011'] + Literacy_11['Female_Urban_2011'] + Literacy_11['Person_Urban_2011']) / 3
Literacy_11['Avg_Rural_2011'] =  (Literacy_11['Male_Rural_2011'] + Literacy_11['Female_Rural_2011'] + Literacy_11['Person_Rural_2011']) / 3
Literacy_11['Avg'] = (Literacy_11['Avg_Urban_2011'] + Literacy_11['Avg_Rural_2011']) / 2

result_2A = pd.read_sql(query_2, engine)
Lit_cor_11 = result_2A['Voters_turnout_per'].corr(Literacy_11['Avg'])

In [49]:
result_2A

,state,Voters_turnout_per
0,Nagaland,16.582239
1,Daman & Diu,14.369805
2,Lakshadweep,12.168315
3,Meghalaya,12.150354
4,Arunachal Pradesh,11.342828
5,Goa,10.728347
6,Mizoram,9.006646
7,Manipur,7.909300
8,Odisha,7.846199
9,Assam,6.959965


In [50]:
if 'Avg' in Literacy_11.columns and 'Voters_turnout_per' in result_2A.columns :
    print(f'Correlation between Votes on GDP percentage and Voter Turnout percentage: {Lit_cor_11}')
else :
    print('Columns for calculating correlation nit found in the result set.')

Correlation between Votes on GDP percentage and Voter Turnout percentage: -0.3686113902033422


In [51]:
Literacy_23

,State,Male,Female,Avg
0,NaN,Literacy Rate %,Literacy Rate %,Literacy Rate %
1,A&N islands,90.11,81.84,86.27
2,Andhra Pradesh,73.4,59.5,66.4
3,Arunachal Pradesh,73.69,59.57,66.95
4,Assam,90.1,81.2,85.9
5,Bihar,79.7,60.5,70.9
6,Chandigarh,90.54,81.38,86.43
7,Chhattisgarh,85.4,68.7,77.3
8,Dadra and Nagar Haveli,86.46,65.93,77.65
9,Daman & Diu,91.48,79.59,87.07


In [52]:
Literacy_23.drop([0], inplace = True)

result_2B = pd.read_sql(query_2, engine)
Lit_cor_23 = result_2B['Voters_turnout_per'].corr(Literacy_23['Avg'].astype(float))

In [53]:
if 'Avg' in Literacy_23.columns and 'Voters_turnout_per' in result_2B.columns :
    print(f'Correlation between Literacy percentage and Voter Turnout percentage After the Election: {Lit_cor_23}')
else :
    print('Columns for calculating correlation nit found in the result set.')

Correlation between Literacy percentage and Voter Turnout percentage After the Election: -0.18326203827614995


In [112]:
GDP_TB27.to_excel('GDP_TB1.xlsx', index = True)